In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from huggingface_hub import notebook_login
# notebook_login()

In [3]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



1650


/tmp/ipykernel_7848/1080511774.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_7848/1080511774.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [4]:
# from datasets import Dataset, load_metric

In [5]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [6]:
len(train)

143

In [7]:
train['label'].value_counts()

label
0    48
2    48
1    47
Name: count, dtype: int64

In [8]:
test['label'].value_counts()

label
0    555
2    552
1    543
Name: count, dtype: int64

In [9]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [10]:
train['embeddings'] = train['text'].apply(model.encode)
test['embeddings'] = test['text'].apply(model.encode)

In [11]:
test

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder,embeddings
56,f1be5f4addf30f47b1d915ca708566bf49beb7f8,ReactiveX-RxJava,Non-blocking implementation of ScheduledObserv...,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.025846384, 0.039176498, -0.08236753, 0.068..."
500,fbf70d4fbfd01a5f7f5798c32d61aad97b7ca540,drools,fix test using using no longer existing- Conse...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.011205062, 0.07077852, -0.01843579, 0.0759..."
1577,40622f0031683d55f12f476f07076021f64bf5b2,aeshell$aesh,changed how the inputstream is read. using Con...,1,0,0,1,1,1,1,...,0,0,1,0,0,0,0,0,0,"[0.005110185, -0.032678794, -0.046633627, 0.01..."
673,14bc62302f5db980c5c21bcb64af047ca68241b5,restlet-framework-java,- The CLAP connector didn't set the- ex...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.045890793, 0.08196205, -0.0035761734, 0.005..."
1470,7a5b4fb0d911a1e32a1dbf3a0431587836545be3,Vala,Add gdu and gdu-gtk bindings to the build syst...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.10360135, 0.19217695, -0.10708542, 0.00772..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,f958ffcc8a52d92385cc29ab896eaf294d695815,orientdb,Fixed issue on browsing of entire cluster/class--,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.18425353, 0.052729346, -0.08904551, 0.1573..."
856,e25dcdd6b12a965b6eca0c69c847fd0316b1c455,camel,CAMEL-1255: Fixed missing classes in .jar - os...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.024664456, 0.43455666, 0.012716342, 0.0781..."
29,935b63527dacf2c380a53ce83ca5b1d06aadca75,ReactiveX-RxJava,Baseline Performance Tests--Start of suite of ...,2,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.24815735, 0.108282186, -0.101496935, -0.09..."
249,e931ef7e6e360b1a89e3f0d97fbc8332852b8dcd,intellij-community,move suppress/settings intention down- (IDEA-7...,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.20967633, -0.03520218, -0.07947293, 0.1012..."


In [12]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [13]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [14]:
from sklearn import metrics
predicted = LR.predict(X_test)

print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.6298235640362976
Logistic Regression Recall: 0.6303030303030303
Logistic Regression Recall: 0.6288470956970179
Logistic Regression Accuracy: 0.6303030303030303
